# 정확도  측정

## 개발 환경 준비

In [1]:
import torch
from PIL import Image
import numpy as np
from numpy.linalg import norm

In [5]:
from library import arcface_onnx
from library import model_zoo
from library import face_analysis
from library import model_func
import onnxruntime

## 모델 준비

In [7]:
app = model_func.model_prepare('models')
app

/Users/namsujin/ai_model/face_model
input mean and std: 127.5 127.5
find model: /Users/namsujin/ai_model/face_model/models/glintr100.onnx recognition
find model: /Users/namsujin/ai_model/face_model/models/scrfd_10g_bnkps.onnx detection
set det-size: (640, 640)


## 모델 실행

In [5]:
is_gray = False
predicts = [] 
# 이미지 파일이 있는 폴더 경로
root = '/Users/ss0510s/korean_test_dataset/esrgan_Low/image/'
# 1:1 text 파일
with open('/Users/ss0510s/korean_test_dataset/LOW/pairs.txt') as f:
    pairs_lines = f.readlines()[0:]

# 모델 실행
with torch.no_grad():
    for i in range(200): # pair 개수 만큼 반복
        p = pairs_lines[i].replace('\n', '').split('\t') # pair을 split하여 리스트로 저장
        # 이미지 경로 저장
        # 동일 인물
        if 3 == len(p):
            sameflag = 1
            name1 = p[0] + '/' + p[0] + '_' + '{:}.jpg'.format(int(p[1]))
            name2 = p[0] + '/' + p[0] + '_' + '{:}.jpg'.format(int(p[2]))
        # 동일하지 않은 인물
        elif 4 == len(p):
            sameflag = 0
            name1 = p[0] + '/' + p[0] + '_' + '{:}.jpg'.format(int(p[1]))
            name2 = p[2] + '/' + p[2] + '_' + '{:}.jpg'.format(int(p[3]))
        else:
            raise ValueError("WRONG LINE IN 'pairs.txt! ")
        # 이미지 파일 open
        with open(root + name1, 'rb') as f:
            img1 =  Image.open(f)
            img1_arr = np.asarray(img1)
            im1 = Image.fromarray((img1_arr * 255).astype(np.uint8))
            rgb_arr1 = np.asarray(im1.convert('RGB'))  
        with open(root + name2, 'rb') as f:
            img2 =  Image.open(f)
            img2_arr = np.asarray(img2)
            im2 = Image.fromarray((img2_arr * 255).astype(np.uint8))
            rgb_arr2 = np.asarray(im2.convert('RGB')) 
        
        # 모델 실행
        res = app.get(rgb_arr1)
        res2 = app.get(rgb_arr2)
        
        try:
            # 임베딩 값 추출
            f1 = res[0].embedding
            f2 = res2[0].embedding
            
            print(f'진행중 {i}')
            # 두 임베딩값 비교 : 유사할 수록 1에 근접
            distance = f1.dot(f2) / (np.linalg.norm(f1) * np.linalg.norm(f2) + 1e-5)
        
        # detection이 되지 않은 경우
        except IndexError as e:
            if(sameflag == 1): # 동일 인물인 경우 default 값
                distance = 0
            elif (sameflag == 0): # 동일 인물이 아닌 경우 default 값
                distance = 1
            print(e)
        
        # 두 파일의 이름, distance, sameflag를 리스트에 저장
        practice = []
        practice.append(name1)
        practice.append(name2)
        practice.append(distance)
        practice.append(sameflag)
        predicts.append(practice)

진행중 0
진행중 1
진행중 2
진행중 3
진행중 4
진행중 5
진행중 6
진행중 7
진행중 8
진행중 9
진행중 10
진행중 11
진행중 12
진행중 13
진행중 14
진행중 15
진행중 16
진행중 17
진행중 18
진행중 19
진행중 20
진행중 21
진행중 22
진행중 23
진행중 24
진행중 25
진행중 26
진행중 27
진행중 28
진행중 29
진행중 30
진행중 31
진행중 32
진행중 33
진행중 34
진행중 35
진행중 36
진행중 37
진행중 38
진행중 39
진행중 40
진행중 41
진행중 42
진행중 43
진행중 44
진행중 45
진행중 46
진행중 47
진행중 48
진행중 49
진행중 50
진행중 51
진행중 52
진행중 53
진행중 54
진행중 55
진행중 56
진행중 57
진행중 58
진행중 59
진행중 60
진행중 61
진행중 62
진행중 63
진행중 64
진행중 65
진행중 66
진행중 67
진행중 68
진행중 69
진행중 70
진행중 71
진행중 72
진행중 73
진행중 74
진행중 75
진행중 76
진행중 77
진행중 78
진행중 79
진행중 80
진행중 81
진행중 82
진행중 83
진행중 84
진행중 85
진행중 86
진행중 87
진행중 88
진행중 89
진행중 90
진행중 91
진행중 92
진행중 93
진행중 94
진행중 95
진행중 96
진행중 97
진행중 98
진행중 99
진행중 100
진행중 101
진행중 102
진행중 103
진행중 104
진행중 105
진행중 106
진행중 107
진행중 108
진행중 109
진행중 110
진행중 111
진행중 112
진행중 113
진행중 114
진행중 115
진행중 116
진행중 117
진행중 118
진행중 119
진행중 120
진행중 121
진행중 122
진행중 123
진행중 124
진행중 125
진행중 126
진행중 127
list index out of range
진행중 129
진행중 130
진행중 131
진행중 132
진행중 133
진행중 134
진행중 135
진행중 13

## best threshold 추출 함수

* thresholds : 임의의 threshold = np.arange(-1.0, 1.0, 0.005)   
* predicts : 모델 실행 결과 리스트
* t : 임의의 test index를 담은 리스트

**predicts에서 index가 t[]에 있는 값만 비교하여 최적의 threshold를 찾아냄**

In [7]:
def find_best_threshold(thresholds, predicts, t):
    best_threshold = best_acc = 0
    
    for threshold in thresholds:
        y_true = [] # 실제 값
        y_predict = [] # 예측 값
        
        for idx, d in enumerate(predicts):
            # index가 t에 있는 값만 비교
            if idx in t:
                # threshold보다 distance가 크면 동일 인물로 출력
                same = 1 if float(d[2]) > threshold else 0
                y_predict.append(same) # 예측값 저장
                y_true.append(int(d[3])) # 실제값 저장
        # 예측값과 실제값 비교
        y_true = np.array(y_true)
        y_predict = np.array(y_predict)
        accuracy = 1.0 * np.count_nonzero(y_true == y_predict) / len(y_true) # 정확도 측정
        
        # 정확도 측정 후 accuracy가 이전보다 높으면 threshold 업데이트
        if accuracy >= best_acc:
            best_acc = accuracy
            best_threshold = threshold
    return best_threshold

## 모델 성능 측정 함수

* threshold : 기준
* diff : 모델 실행 결과가 담긴 리스트

In [8]:
def eval_acc(threshold, diff):
    y_true = [] # 정답값
    y_predict = [] # 예측값
    tp = 0
    tn = 0
    fp = 0
    fn = 0
    for idx, d in enumerate(diff):
        # threshold보다 distance가 크면 동일인물로 출력
        same = 1 if float(d[2]) > threshold else 0
        y_predict.append(same) # 예측값 저장
        y_true.append(int(d[3])) # 정답값 저장
        
        # 값 비교
        if(same == 1 and int(d[3])== 1): # 예측값과 실제값이 동일 인물
            tp = tp + 1
        elif(same == 0 and int(d[3])== 0): # 예측값과 실제값이 다른 인물
            tn = tn + 1
        elif(same == 1 and int(d[3])== 0): # 예측값은 동일인물, 실제는 다른 인물
            fp = fp + 1
        if(same == 0 and int(d[3])== 1): # 예측값은 다른 인물, 실제는 동일 인물
            fn = fn + 1
    
    # numpy로 변환
    y_true = np.array(y_true)
    y_predict = np.array(y_predict)

    # recall, precision 계산
    rec = float(tp / (tp + fn))
    pre = float( tp / (tp + fp ))
    
    # accuracy 계산
    accuracy = 1.0 * np.count_nonzero(y_true == y_predict) / len(y_true)
    return rec, pre, accuracy

## 모델 성능 측정

In [6]:
# 초기화
accuracy = 0
thd = 0
thresholds = np.arange(-1.0, 1.0, 0.005)
precision = 0
recall = 0
indices = 0
t = [1, 2, 3,4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20,
    101, 102, 103,104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120]

In [9]:
# 함수 실행
best_thresh = find_best_threshold(thresholds, predicts, t)

recall, precision, accuracy = eval_acc(best_thresh, predicts)

print(recall, precision, accuracy)

0.78 0.6902654867256637 0.715
